In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import scipy as sp
from collections import defaultdict
from itertools import product
import gurobipy as gb
from gurobipy import GRB
from tqdm import tqdm
from itertools import chain
from itertools import permutations

repeats = json.load(open('../cleanerdata/repeats.json'))
repeats = {int(k):v for k,v in repeats.items()}

locdem = pd.read_excel('../cleanerdata/locdem.xlsx')

q = locdem['Number of pallets'].to_numpy().astype(float).tolist()
q = [0] + q
q.extend([v['dem'] for v in repeats.values()])

locs = pd.read_excel('../cleanerdata/locations.xlsx')
longs, lats = locs['long'].to_numpy().tolist(), locs['lat'].to_numpy().tolist()
longs.extend([longs[v['map']] for v in repeats.values()])
lats.extend([lats[v['map']] for v in repeats.values()])

distmat = pd.read_json('../cleanerdata/distmat.json').to_numpy()
timemat = pd.read_json('../cleanerdata/timemat.json').to_numpy()
def get(i):
    try:
        i = repeats[i]['map']
    except KeyError:
        pass
    return i

reverse_mapping = defaultdict(list)
for k,v in repeats.items():
    reverse_mapping[v['map']].append(k)
for k in reverse_mapping:
    reverse_mapping[k].append(k)

mappings = list(chain(*[permutations(ls,2) for ls in reverse_mapping.values()]))

def cost(i,j):
    if (i,j) in mappings:
        return 1e+8
    i,j = get(i), get(j)
    return distmat[i,j]

def time(i,j):
    i,j = get(i), get(j)
    return timemat[i,j]

def location(i):
    return longs[i], lats[i]

In [2]:
clusters = json.load(open('./clusters.json'))
list(map(lambda x: len(x), clusters))

[7, 6, 7, 7, 7, 7, 7, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]

In [3]:
Q = 9
iter_count = 0
primary_models = []
Ls = []

for cluster in clusters:
    Neighbours = list(sorted(cluster))
    all_nodes = [0] + Neighbours.copy()
    v = len(all_nodes)
    V = list(range(v))
    N = V.copy()
    N.remove(0)
    
    def Del(i):
        F = V.copy()
        F.remove(i)
        return F
    
    m = gb.Model()
    x = m.addVars(v,v, vtype=GRB.BINARY, name='x')
    m.setObjective(sum([cost(all_nodes[i], all_nodes[j]) * x[i,j] for i,j in product(V,V)]))
    for i in N:
        m.addConstr(sum([x[i,j] for j in Del(i)]) == 1)
    for j in N:
        m.addConstr(sum([x[i,j] for i in Del(j)]) == 1)
        
    l = m.addVar(lb=0.0, vtype=GRB.INTEGER, name='L')
    Ls.append(l)
    m.addConstr(sum([x[0,j] for j in Del(0)]) == l)
    
    u = m.addVars(V, vtype=GRB.CONTINUOUS, name='u')
    for i,j in filter(lambda tup: tup[0] != tup[1], product(N,N)):
        m.addConstr(u[i] - u[j] + Q*x[i,j] <= Q - q[j])
    for i in V:
        m.addConstr(u[i] >= q[i])
        
    for pair in mappings:
        if pair[0] in cluster and pair[1] in cluster:
            pair = Neighbours.index(pair[0])+1, Neighbours.index(pair[1])+1
            #print(Neighbours, pair)
            for length in range(1,6):
                all_groups = permutations(N,length)
                for group in all_groups:
                    if pair[0] in group or pair[1] in group:
                        continue
                    group = [pair[0]] + list(group) + [pair[1]]
                    edges = [(group[i], group[i+1]) for i in range(len(group) - 1)]
                    expr = 0
                    for edge in edges:
                        expr += x[edge]
                    m.addConstr(expr <= length)
    
    primary_models.append(m)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-12


In [4]:
for model in tqdm(primary_models):
    model.optimize();

  0%|                                                    | 0/26 [00:00<?, ?it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 65 rows, 73 columns and 240 nonzeros
Model fingerprint: 0xea1036e7
Variable types: 8 continuous, 65 integer (64 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [4e+03, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 394600.00000
Presolve removed 9 rows and 24 columns
Presolve time: 0.00s
Presolved: 56 rows, 49 columns, 210 nonzeros
Variable types: 7 continuous, 42 integer (42 binary)
Found heuristic solution: objective 199353.00000

Root relaxation: objective 5.074589e+04, 18 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 50745.8889    0   10 199353.000 50745.8889  74.5

  4%|█▋                                          | 1/26 [00:00<00:14,  1.69it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 50 rows, 57 columns and 176 nonzeros
Model fingerprint: 0x7984830f
Variable types: 7 continuous, 50 integer (49 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [5e+03, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 780802.00000
Presolve removed 8 rows and 21 columns
Presolve time: 0.00s
Presolved: 42 rows, 36 columns, 150 nonzeros
Variable types: 6 continuous, 30 integer (30 binary)
Found heuristic solution: objective 342884.00000

Root relaxation: objective 1.529766e+05, 18 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 152976.556    0    9 342884.000 152976.556  55.4

  8%|███▍                                        | 2/26 [00:00<00:10,  2.37it/s]

...

 92%|███████████████████████████████████████▋   | 24/26 [00:15<00:00,  2.26it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 715 rows, 73 columns and 3500 nonzeros
Model fingerprint: 0x9f49b655
Variable types: 8 continuous, 65 integer (64 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+02, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 392158.00000
Presolve removed 9 rows and 26 columns
Presolve time: 0.01s
Presolved: 706 rows, 47 columns, 5424 nonzeros
Variable types: 7 continuous, 40 integer (40 binary)
Found heuristic solution: objective 123128.00000

Root relaxation: objective 3.311333e+04, 27 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 33113.3333    0   11 123128.000 33113.3333  

 96%|█████████████████████████████████████████▎ | 25/26 [00:16<00:00,  1.59it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1365 rows, 73 columns and 6760 nonzeros
Model fingerprint: 0x34095a6f
Variable types: 8 continuous, 65 integer (64 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [6e+03, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 715811.00000
Presolve removed 857 rows and 28 columns
Presolve time: 0.03s
Presolved: 508 rows, 45 columns, 3196 nonzeros
Variable types: 7 continuous, 38 integer (38 binary)
Found heuristic solution: objective 423669.00000

Root relaxation: objective 1.912890e+05, 22 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 191289.000    0   12 423669.000 191289.00

100%|███████████████████████████████████████████| 26/26 [00:17<00:00,  1.47it/s]


In [5]:
model_solutions = {}
for i,model in enumerate(primary_models):
    soln = json.loads(model.getJSONSolution())
    model_solutions[i] = soln
json.dump(model_solutions, open('./primary_model_solutions.json', 'w'), indent=4)

In [6]:
s = 0
for model in primary_models:
    s += model.ObjVal
s

6827261.0

In [7]:
from itertools import permutations
K = [1,2,3]
print(*permutations(K,2))

(1, 2) (1, 3) (2, 1) (2, 3) (3, 1) (3, 2)
